# word2vec 단어 간 유사도 체크
- 최대한 육아를 주제로 한 말뭉치를 모아보았다. (Total 2,755 documents)
- 우리가 원하는 의도에 일치하는 단어 간 유사도 결과가 나오는지를 확인하는 것이 목적이다.

## Import

In [1]:
import pandas as pd
import numpy as np
import re

from konlpy.tag import Mecab
from konlpy.tag import Komoran
from gensim.models import Word2Vec

### Datas

In [2]:
df1 = pd.read_csv("data/maeili.csv", encoding='utf-8')
df2 = pd.read_csv("data/baby.csv", encoding='utf-8')
df3 = pd.read_csv("data/toys.csv", encoding='utf-8')
df4 = pd.read_csv("data/seoulchildcare.csv", encoding='utf-8')
df5 = pd.read_csv("data/babynews_DLC.csv", encoding='utf-8')
df6 = pd.read_csv("data/ildongfoodis.csv", encoding='utf-8')
df7 = pd.read_csv("data/momsmagazine.csv", encoding='utf-8')
df8 = pd.read_csv("data/ange.csv", encoding='utf-8')
df9 = pd.read_csv("data/babytimes.csv", encoding='utf-8')
df10 = pd.read_csv("data/daekyo.csv", encoding='utf-8')
df11 = pd.read_csv("data/mom&enfant.csv", encoding='utf-8')
df12 = pd.read_csv("data/namyang.csv", encoding='utf-8')

### Concatenate Dataframe

In [3]:
df = pd.concat([df1.text, df2.text, df3.text, df4.text, df5.text,
                df6.text, df7.text, df8.text, df9.text, df10.text,
                df11.text, df12.text])
df = pd.DataFrame(df).reset_index(drop=True)
df.tail()

,text
27050,아토피에 유전적인 요인이 작용한다는 사실 알고 계시나요 유전적인 요인 뿐 아니라 아...
27051,1st 시기에 따라 필요한 영양소를 충분히 뱃속의 아이는 엄마가 먹는 것을 함께 먹...
27052,분만 여성만이 누릴 수 있는 특권 대부분의 여성들은 임신과 분만이라는 과정을 경험하...
27053,최근 전자파가 건강상의 문제를 일으킬 수 있다는 견해가 많아지면서 임신부들 또한 가...
27054,임신 기간을 설명할 때 헷갈리는 경우가 많다 책을 볼 때 의사의 설명을 들을 때 정...


### Preprocessing

In [4]:
def cleaningText(text):
    text = re.sub('&nbsp;|\xa0|\u200b|\r|\n|\t', ' ', text)
    text = re.sub('<.+>|【.+】|\[.+\]', ' ', text)
    text = re.sub('[-_=+,/\?;:^$.@*\"※~&%ㆍ!”“’「」『』〈〉\\‘|\(\)\{\}\[\]\<\>`\'…《》]', ' ', text)
    text = re.sub('[㈜×–～→▪‧•∘✽○□◇△▽▷◁☆◈◐▶▼▲◆★♦■♥️●♠🖤]', ' ', text)
    text = re.sub('[①②③④⑤⑥⑦⑧⑨⑩⑪⑫⑬⑭⑮⑯⑰⑱⑲⑳]', ' ', text)
    text = re.sub('[➊❷➌❹❺❻❼❽❾❿]', ' ', text)
    text = re.sub('http.+com|http.+co|http.+kr|www.+kr|www.+com|www.+net|http.+net', ' ', text)
    text = re.sub("\s.+\s기자", '', text)
    text = re.sub('ⓒ\S+\s|㈜\S+\s|홈페이지\S+\s|전화\S+\s|대표전화\S+\s|☎\S+\s|http\S+\s', '', text)
    
    text = re.sub('\s+', ' ', text)
    text = text.strip()

    return text


In [5]:
df = df.drop(index=[952, 955, 956, 960, 962, 964, 1165, 1240, 1322])
df = df.drop_duplicates()
df = df.dropna()
df = pd.DataFrame(df).reset_index(drop=True)

In [6]:
df['text'] = df['text'].apply(cleaningText)
df = df.dropna()
df.to_csv('data/text_for_word2vec.csv', index=False, encoding='utf-8')

In [7]:
df

,text
0,6살 남아 입니다 회사 복직 후 3살부터 주중에는 외할머니댁에서 2살 4살 터울의 ...
1,요즘 들어 아빠가 아이에게 다가가려고 하면 엄마한테 갈 거야 하면서 엄마를 유독 찾...
2,둘째가 태어나면 첫째가 힘들 거라는 건 예상하고 있었지만 툭하면 삐치고 울고 좀처럼...
3,20개월 남아가 있고 임신 8개월차로 두 아기의 엄마입니다 첫 아이는 잠자리 독립을...
4,14개월 남자 아기인데 심하게 깨무는 버릇이 있어요 전에는 졸릴 때 주로 물곤 했는...
...,...
25344,가임기 연력 여성의 흡연률 증가와 그 폐해 현대 여성은 다양한 사회활동에 참여하고 ...
25345,아토피에 유전적인 요인이 작용한다는 사실 알고 계시나요 유전적인 요인 뿐 아니라 아...
25346,1st 시기에 따라 필요한 영양소를 충분히 뱃속의 아이는 엄마가 먹는 것을 함께 먹...
25347,분만 여성만이 누릴 수 있는 특권 대부분의 여성들은 임신과 분만이라는 과정을 경험하...


### Add tokenized column

191210 정체불명의 error 발생
```
NotImplementedError: Wrong number or type of arguments for overloaded function 'Tagger_parse'.
  Possible C/C++ prototypes are:
    MeCab::Tagger::parse(MeCab::Model const &,MeCab::Lattice *)
    MeCab::Tagger::parse(MeCab::Lattice *) const
    MeCab::Tagger::parse(char const *)
```

babynews 데이터가 추가된 후 갑자기 발생한 에러인데.. 왜인지 모르겠지만 해당 부분 데이터프레임만 따로 tokenize하는 것은 문제없이 돌아간다.  
따라서 번거롭지만 분리하여 tokenizing 한 후에 합치기로 하였다.  

-> 해결.. `NaN` value 문제였다(어이없게도 NaN 값이 중간에 섞여있었음;). 누락 데이터를 잘 체크하는 습관을 갖도록하자.

In [8]:
def tokenize(text):
    tokenizer = Mecab()
    text = tokenizer.morphs(text)
#     text = ' '.join(text)
    return text

In [9]:
%%time
df['tokenized'] = df['text'].apply(tokenize)
df.tail()

CPU times: user 1min, sys: 29.2 s, total: 1min 29s
Wall time: 1min 29s


,text,tokenized
25344,가임기 연력 여성의 흡연률 증가와 그 폐해 현대 여성은 다양한 사회활동에 참여하고 ...,"[가, 임기, 연력, 여성, 의, 흡연, 률, 증가, 와, 그, 폐해, 현대, 여성..."
25345,아토피에 유전적인 요인이 작용한다는 사실 알고 계시나요 유전적인 요인 뿐 아니라 아...,"[아토피, 에, 유전, 적, 인, 요인, 이, 작용, 한다는, 사실, 알, 고, 계..."
25346,1st 시기에 따라 필요한 영양소를 충분히 뱃속의 아이는 엄마가 먹는 것을 함께 먹...,"[1, st, 시기, 에, 따라, 필요, 한, 영양소, 를, 충분히, 뱃속, 의, ..."
25347,분만 여성만이 누릴 수 있는 특권 대부분의 여성들은 임신과 분만이라는 과정을 경험하...,"[분만, 여성, 만, 이, 누릴, 수, 있, 는, 특권, 대부분, 의, 여성, 들,..."
25348,최근 전자파가 건강상의 문제를 일으킬 수 있다는 견해가 많아지면서 임신부들 또한 가...,"[최근, 전자파, 가, 건강, 상, 의, 문제, 를, 일으킬, 수, 있, 다는, 견..."


In [10]:
df.isnull().sum()

text         0
tokenized    0
dtype: int64

## Train Word2Vec

#### WTF error
- df를 csv 형식으로 저장한 후 다시 로드하면 Word2Vec 학습에 인코딩 문제가 생긴다.
- 더 자세히 파고들어보니 list 형식의 column이 csv로 저장할 때 강제로 string 타입으로 형변환 됨.
- 이렇게 형변환 된 column을 Word2Vec 학습에 사용할 시 한글 텍스트가 깨지는 현상이 발생함.. 정확한 원인은 아직 모르겠다.

In [11]:
# df = pd.read_csv('text_for_word2vec.csv', encoding='utf-8')

그러므로 저장 & 로드 과정 생략하고 바로 말뭉치를 numpy ndarray형식으로 변환

In [12]:
corpus = np.array(df.tokenized)
type(corpus)

numpy.ndarray

In [13]:
%%time
model_all = Word2Vec(corpus,
                     size=200,
                     window=6,
                     min_count=5,
                     workers=4,
                     seed=1,
                     sg=1)
model_all.save('data/tokmom_word2vec_all.model')

CPU times: user 8min 55s, sys: 1.08 s, total: 8min 56s
Wall time: 2min 20s


#### 유사어 분석

In [14]:
model_all.wv.similar_by_word("놀이")

[('소꿉', 0.6875901818275452),
 ('공놀이', 0.657809853553772),
 ('불록', 0.6439043283462524),
 ('무너뜨리기', 0.6286779642105103),
 ('기차놀이', 0.6262086629867554),
 ('음률', 0.6185630559921265),
 ('집짓기', 0.6170854568481445),
 ('손유희', 0.6154947280883789),
 ('점토', 0.6150206327438354),
 ('짝짜꿍', 0.6118137240409851)]

In [15]:
model_all.wv.similar_by_word("아이")

[('아기', 0.6730787754058838),
 ('어린아이', 0.5907984375953674),
 ('친해진다', 0.5837182998657227),
 ('바꿔라', 0.5807464122772217),
 ('엄마', 0.5802526473999023),
 ('갓난아기', 0.5787348747253418),
 ('아이라', 0.5763530731201172),
 ('수줍어하', 0.5753364562988281),
 ('어울려야', 0.5752732753753662),
 ('지나갑니다', 0.5706583261489868)]

In [16]:
model_all.wv.similar_by_word("치아")

[('잇몸', 0.7870566248893738),
 ('영구치', 0.7553123235702515),
 ('충치', 0.7464313507080078),
 ('젖니', 0.7303667068481445),
 ('턱뼈', 0.6853368282318115),
 ('어금니', 0.6830192804336548),
 ('앞니', 0.677143394947052),
 ('에나멜질', 0.6670094728469849),
 ('유치', 0.6667609214782715),
 ('치석', 0.663247287273407)]

In [17]:
model_all.wv.similar_by_word("이빨")

[('혓바닥', 0.7488198280334473),
 ('거뭇', 0.7317402362823486),
 ('거스러미', 0.7167705297470093),
 ('허옇', 0.7036391496658325),
 ('갈라진다', 0.6965320706367493),
 ('생채기', 0.6960688829421997),
 ('작아져서', 0.6927667260169983),
 ('돌덩이', 0.691351056098938),
 ('뜯어내', 0.6902056932449341),
 ('그리곤', 0.6883012652397156)]

In [18]:
model_all.wv.similar_by_word("감기")

[('목감기', 0.7839117050170898),
 ('코감기', 0.7689055800437927),
 ('기침감기', 0.7520759105682373),
 ('독감', 0.7218563556671143),
 ('편도염', 0.7058274745941162),
 ('환절기', 0.6969997882843018),
 ('냉방병', 0.6951795816421509),
 ('기관지염', 0.6949224472045898),
 ('호흡기', 0.6901829838752747),
 ('잔병', 0.6875166893005371)]

In [19]:
model_all.wv.similar_by_word("열")

[('고열로', 0.5601767897605896),
 ('고열', 0.5428673028945923),
 ('미열', 0.5291249752044678),
 ('성경', 0.5286558866500854),
 ('들어온다고', 0.5261867642402649),
 ('오한', 0.5185509324073792),
 ('외감', 0.5166741609573364),
 ('사병', 0.5139747858047485),
 ('어지러울', 0.49629321694374084),
 ('심열', 0.4948289692401886)]

In [20]:
model_all.wv.similar_by_word("아파")

[('아프', 0.7680637836456299),
 ('아파서', 0.7262787818908691),
 ('아파하', 0.7010130882263184),
 ('지끈지끈', 0.6951578855514526),
 ('고파도', 0.6796739101409912),
 ('토해서', 0.678600549697876),
 ('아픈데', 0.6783655285835266),
 ('칭얼댄다', 0.6745063662528992),
 ('다쳐', 0.6729034185409546),
 ('먹먹해', 0.6720726490020752)]

In [21]:
model_all.wv.similar_by_word("아프")

[('아파', 0.7680637836456299),
 ('아파하', 0.7122815847396851),
 ('지끈지끈', 0.7047165632247925),
 ('시큰거리', 0.6809597611427307),
 ('먹먹해', 0.6796846389770508),
 ('아파서', 0.6785406470298767),
 ('아픈데', 0.6740100383758545),
 ('결리', 0.673911988735199),
 ('아픈', 0.671587347984314),
 ('주물렀', 0.6704263687133789)]

In [22]:
model_all.wv.similar_by_word("울음")

[('터뜨립니다', 0.683521032333374),
 ('터뜨린', 0.6515367031097412),
 ('터뜨린다', 0.649518609046936),
 ('터뜨릴', 0.6469658613204956),
 ('자지러질', 0.6336624622344971),
 ('지르기', 0.6222484111785889),
 ('옹알이', 0.6205344200134277),
 ('고래고래', 0.6195508241653442),
 ('우렁찬', 0.6156233549118042),
 ('신음', 0.6145595908164978)]

In [23]:
model_all.wv.similar_by_word("남아")

[('여아', 0.8171848058700562),
 ('정상아', 0.5825268030166626),
 ('엘렉트라', 0.5784873366355896),
 ('남근기', 0.5734211206436157),
 ('018', 0.5476497411727905),
 ('내사시', 0.54610276222229),
 ('초산부', 0.5425697565078735),
 ('정상인', 0.5369791984558105),
 ('여자', 0.5365892648696899),
 ('보였으나', 0.5297045707702637)]

In [24]:
model_all.wv.similar_by_word("여아")

[('남아', 0.8171848058700562),
 ('여자', 0.5540707111358643),
 ('아든', 0.5492159128189087),
 ('남자', 0.5310503244400024),
 ('정상아', 0.5236005187034607),
 ('617', 0.520268440246582),
 ('018', 0.5196837782859802),
 ('엘렉트라', 0.508274257183075),
 ('아인지', 0.5071039199829102),
 ('남근기', 0.5036231279373169)]

---

## Only Noun

- 이번에는 말뭉치에서 명사만 추출하여 결과를 비교해보자
- 유사한 키워드 잡기를 목적으로 할 때는 명사만 추출하는 것이 더 적절한 듯

In [25]:
%%time
def extract_noun(text):
    tokenizer = Mecab()
    text = tokenizer.nouns(text)
#     text = ' '.join(text)
    return text
    
df['noun'] = df['text'].apply(extract_noun)
df = df.dropna()

CPU times: user 1min 3s, sys: 29 s, total: 1min 32s
Wall time: 1min 31s


In [26]:
noun_corpus = np.array(df.noun)

In [27]:
%%time
model_noun = Word2Vec(noun_corpus,
                      size=200,
                      window=2,
                      workers=4,
                      min_count=5,
                      seed=1,
                      sg=1)
model_noun.save('data/tokmom_word2vec_only_noun.model')

CPU times: user 1min 56s, sys: 305 ms, total: 1min 56s
Wall time: 32.9 s


In [28]:
model_noun.wv.similar_by_word("육아")

[('양육', 0.6147451400756836),
 ('워킹맘', 0.5736454725265503),
 ('열혈', 0.5688308477401733),
 ('자처', 0.5678771734237671),
 ('할당제', 0.5650680065155029),
 ('노동법', 0.5634746551513672),
 ('바깥일', 0.5617146492004395),
 ('재테크', 0.5529548525810242),
 ('짠돌이', 0.5514470338821411),
 ('독박', 0.5475138425827026)]

In [29]:
model_noun.wv.similar_by_word("시댁")

[('시부모', 0.819999098777771),
 ('처가', 0.7896651029586792),
 ('시어머니', 0.7749943733215332),
 ('친가', 0.7676393985748291),
 ('장모님', 0.7660458087921143),
 ('시누이', 0.7597363591194153),
 ('친정', 0.7590876221656799),
 ('친정어머니', 0.7583170533180237),
 ('며느리', 0.7528148889541626),
 ('식구', 0.739051103591919)]

In [30]:
model_noun.wv.similar_by_word("놀이")

[('놀잇감', 0.6733739376068115),
 ('공놀이', 0.6566992402076721),
 ('소꿉', 0.6336125731468201),
 ('기차놀이', 0.6302005648612976),
 ('점토', 0.6167163848876953),
 ('지점토', 0.6129889488220215),
 ('줄다리기', 0.6077327728271484),
 ('도미노', 0.6064156293869019),
 ('종이접기', 0.6053587794303894),
 ('비석', 0.602632999420166)]

In [31]:
model_noun.wv.similar_by_word("아이")

[('어린아이', 0.6893014311790466),
 ('말자', 0.6803719997406006),
 ('갓난아기', 0.6760104894638062),
 ('아기', 0.6680408716201782),
 ('지레짐작', 0.6662882566452026),
 ('무의식중', 0.6636631488800049),
 ('아무거', 0.6632058620452881),
 ('안달복달', 0.6584646701812744),
 ('딴청', 0.6565935611724854),
 ('부리기', 0.6549821496009827)]

In [32]:
model_noun.wv.similar_by_word("치아")

[('잇몸', 0.7748003005981445),
 ('영구치', 0.7499099373817444),
 ('충치', 0.7321614027023315),
 ('젖니', 0.7302331328392029),
 ('유치', 0.6928586959838867),
 ('어금니', 0.69121915102005),
 ('과잉치', 0.6696228981018066),
 ('법랑질', 0.6678387522697449),
 ('치근', 0.6618191003799438),
 ('앞니', 0.6617239117622375)]

In [33]:
model_noun.wv.similar_by_word("이빨")

[('혓바닥', 0.7992572784423828),
 ('풀칠', 0.7745968103408813),
 ('거스러미', 0.7719434499740601),
 ('거립니다', 0.7702248096466064),
 ('옷깃', 0.7634695768356323),
 ('생채기', 0.7633864283561707),
 ('벙어리', 0.7606757879257202),
 ('덧', 0.7596206665039062),
 ('껍데기', 0.752647876739502),
 ('아랫입술', 0.7522115707397461)]

In [34]:
model_noun.wv.similar_by_word("감기")

[('코감기', 0.7475954294204712),
 ('목감기', 0.7324339747428894),
 ('기침감기', 0.7220199704170227),
 ('편도염', 0.705484926700592),
 ('기관지염', 0.7047243714332581),
 ('냉방병', 0.691491961479187),
 ('병치레', 0.6910607814788818),
 ('잔병치레', 0.6803764700889587),
 ('환절기', 0.6791157126426697),
 ('마른기침', 0.678935170173645)]

In [35]:
model_noun.wv.similar_by_word("열")

[('고열', 0.5995689630508423),
 ('고열로', 0.598488450050354),
 ('오한', 0.5892696380615234),
 ('미열', 0.566575825214386),
 ('불덩이', 0.5622368454933167),
 ('허열', 0.5592338442802429),
 ('사병', 0.5574789047241211),
 ('일사병', 0.5371001362800598),
 ('발열', 0.5334112644195557),
 ('몸살감기', 0.5328003168106079)]

In [36]:
model_noun.wv.similar_by_word("아파")

[('열꽃', 0.8223117589950562),
 ('참기', 0.8178750276565552),
 ('어린애', 0.8170937299728394),
 ('울상', 0.8074944019317627),
 ('청진', 0.8015746474266052),
 ('헛소리', 0.8009097576141357),
 ('생채기', 0.8004393577575684),
 ('불덩이', 0.7994793057441711),
 ('피멍', 0.799109160900116),
 ('황달기', 0.7982150316238403)]

In [37]:
model_noun.wv.similar_by_word("울음")

[('지르기', 0.6716539263725281),
 ('놀람', 0.6687363982200623),
 ('큰소리', 0.6645467281341553),
 ('비명', 0.654954731464386),
 ('세요', 0.6495398283004761),
 ('아달라', 0.6392082571983337),
 ('신음', 0.6357528567314148),
 ('괴성', 0.6203086972236633),
 ('무안', 0.618242621421814),
 ('낼', 0.6176371574401855)]

In [38]:
model_noun.wv.similar_by_word("남아")

[('여아', 0.8225380182266235),
 ('생시', 0.6644451022148132),
 ('정상아', 0.651761531829834),
 ('초혼', 0.6464936137199402),
 ('음경', 0.6348172426223755),
 ('확연', 0.6345064640045166),
 ('정류', 0.6334263682365417),
 ('외반슬', 0.6323204636573792),
 ('자폐아', 0.6320158839225769),
 ('조숙', 0.630962073802948)]

In [39]:
model_noun.wv.similar_by_word("여아")

[('남아', 0.8225380182266235),
 ('체구', 0.6191718578338623),
 ('남학생', 0.6152935028076172),
 ('생시', 0.6109334826469421),
 ('곱슬머리', 0.607616126537323),
 ('항문기', 0.6026288866996765),
 ('조숙', 0.6004122495651245),
 ('승우', 0.597633957862854),
 ('초혼', 0.5976155996322632),
 ('가슴둘레', 0.5970245599746704)]

In [40]:
model_noun.wv.similar_by_word("여자")

[('남자', 0.844328761100769),
 ('여아', 0.5794215202331543),
 ('남아', 0.5660580396652222),
 ('조신', 0.5549825429916382),
 ('금성', 0.5498636960983276),
 ('아이든', 0.5464881658554077),
 ('남학생', 0.5455925464630127),
 ('여탕', 0.5252121686935425),
 ('오이디푸스', 0.5185407996177673),
 ('엘렉트라', 0.5105907917022705)]

In [41]:
model_noun.wv.similar_by_word("남자")

[('여자', 0.8443288207054138),
 ('금성', 0.6193404197692871),
 ('조신', 0.6130093932151794),
 ('아이든', 0.602680504322052),
 ('얌전', 0.6016589403152466),
 ('남학생', 0.5876859426498413),
 ('여탕', 0.5767632126808167),
 ('여아', 0.5758135318756104),
 ('태생', 0.5740955471992493),
 ('남아', 0.573989987373352)]

In [42]:
model_noun.wv.similar_by_word("유모차")

[('디럭스', 0.7052377462387085),
 ('절충', 0.6578075885772705),
 ('타보', 0.6569042205810547),
 ('잉글', 0.6366807222366333),
 ('휴대용', 0.6336427927017212),
 ('범용', 0.6298631429672241),
 ('퀴니', 0.6164275407791138),
 ('레시', 0.6130141019821167),
 ('휠체어', 0.6098400354385376),
 ('카시트', 0.6092009544372559)]

In [43]:
model_noun.wv.similar_by_word("카시트")

[('컨버터블', 0.6584720611572266),
 ('안전벨트', 0.6522437334060669),
 ('캐리어', 0.6180393695831299),
 ('뒷좌석', 0.6131865978240967),
 ('부스터', 0.6111732721328735),
 ('순성', 0.610962986946106),
 ('장착', 0.6108025312423706),
 ('유모차', 0.6092009544372559),
 ('브라이', 0.6090240478515625),
 ('콩코드', 0.6050165891647339)]

In [81]:
model_noun.wv.most_similar(positive=["유모차", "카시트"])

[('디럭스', 0.7086952924728394),
 ('휴대용', 0.6859462261199951),
 ('범용', 0.6835785508155823),
 ('절충', 0.672086775302887),
 ('캐리어', 0.669406533241272),
 ('페도라', 0.6677770614624023),
 ('컨버터블', 0.6605300903320312),
 ('타보', 0.659294605255127),
 ('요람', 0.65788733959198),
 ('콩코드', 0.6575262546539307)]

In [90]:
model_noun.wv.most_similar(positive=["중고", "거래"])

[('반품', 0.7995040416717529),
 ('손품', 0.7655367255210876),
 ('최저가', 0.7590097784996033),
 ('중개', 0.748986005783081),
 ('판매자', 0.7404036521911621),
 ('중고차', 0.7399103045463562),
 ('반납', 0.7365485429763794),
 ('계약서', 0.734481930732727),
 ('귀책', 0.7343562841415405),
 ('통관', 0.7323704957962036)]

In [97]:
model_noun.wv.most_similar(positive=["여아", "중고"])

[('필독', 0.6952844262123108),
 ('기서', 0.6877860426902771),
 ('충동구매', 0.6863868236541748),
 ('유아복', 0.6862215995788574),
 ('세용', 0.6747269630432129),
 ('너그', 0.6695094108581543),
 ('남아', 0.6640238761901855),
 ('손품', 0.6635791063308716),
 ('연령층', 0.6616824865341187),
 ('킬로', 0.6613768339157104)]

---

## Keywords (Noun + Verb + Adj)

- 마지막으로 말뭉치에서 명사, 동사, 형용사까지 추출한 결과를 확인

In [44]:
%%time
def extract_keywords(text):
    tokenizer = Mecab()
    pos_list = tokenizer.pos(text)
    keywords_list = list(filter(lambda w: 'NNG' in w or \
                                          'NNP' in w or \
                                          'VA' in w or \
                                          'VV' in w , pos_list))
    keywords = list(map(lambda w: w[0], keywords_list))
    return keywords

df['keywords'] = df['text'].apply(extract_keywords)
df = df.dropna()

CPU times: user 1min 4s, sys: 29.1 s, total: 1min 33s
Wall time: 1min 32s


In [45]:
keywords_corpus = np.array(df.keywords)

In [124]:
%%time
model_keywords = Word2Vec(keywords_corpus,
                          size=200,
                          window=3,
                          min_count=3,
                          workers=4,
                          seed=1,
                          sg=1)
model_keywords.save('data/tokmom_word2vec_keywords.model')

CPU times: user 2min 36s, sys: 294 ms, total: 2min 36s
Wall time: 44 s


In [125]:
model_keywords.wv.similar_by_word("육아")

[('양육', 0.6057778596878052),
 ('독박', 0.6025557518005371),
 ('워킹맘', 0.5992558598518372),
 ('열혈', 0.587739884853363),
 ('아지', 0.5754351615905762),
 ('신박', 0.568577766418457),
 ('지원군', 0.5676036477088928),
 ('똑소리', 0.5594679117202759),
 ('자처', 0.5573272705078125),
 ('떠맡', 0.5527890920639038)]

In [126]:
model_keywords.wv.similar_by_word("시댁")

[('시부모', 0.8208571076393127),
 ('처가', 0.7792154550552368),
 ('친정', 0.7738823890686035),
 ('본가', 0.7567782998085022),
 ('장모님', 0.7299110889434814),
 ('며느리', 0.7289063334465027),
 ('대소사', 0.7286662459373474),
 ('친정어머니', 0.7283321619033813),
 ('시어머니', 0.7175921201705933),
 ('친가', 0.7131330966949463)]

In [127]:
model_keywords.wv.similar_by_word("놀이")

[('놀잇감', 0.676058292388916),
 ('공놀이', 0.6560580134391785),
 ('소꿉', 0.6478983163833618),
 ('재미있', 0.6389245390892029),
 ('기차놀이', 0.6344831585884094),
 ('무너뜨리기', 0.6282132267951965),
 ('놀', 0.6263772249221802),
 ('딱지치기', 0.6202889680862427),
 ('실뜨기', 0.6116752624511719),
 ('손인', 0.5988839864730835)]

In [128]:
model_keywords.wv.similar_by_word("아이")

[('눈치채', 0.6995992660522461),
 ('지레짐작', 0.6972516775131226),
 ('아기', 0.6958725452423096),
 ('안달', 0.69270920753479),
 ('어린아이', 0.6847321391105652),
 ('유별나', 0.6846099495887756),
 ('뒤돌아보', 0.6832249760627747),
 ('아이라', 0.6819329261779785),
 ('돌쟁이', 0.6784884929656982),
 ('말자', 0.6777140498161316)]

In [129]:
model_keywords.wv.similar_by_word("치아")

[('영구치', 0.7670557498931885),
 ('잇몸', 0.7630598545074463),
 ('젖니', 0.7545617818832397),
 ('충치', 0.737262487411499),
 ('앞니', 0.70076584815979),
 ('어금니', 0.6859873533248901),
 ('법랑질', 0.6776264905929565),
 ('윗니', 0.6743996739387512),
 ('치조골', 0.6739828586578369),
 ('과잉치', 0.6692478656768799)]

In [130]:
model_keywords.wv.similar_by_word("이빨")

[('영인', 0.8284428715705872),
 ('혓바닥', 0.8143986463546753),
 ('거스러미', 0.8136392831802368),
 ('역겹', 0.7993366718292236),
 ('갉', 0.7961187362670898),
 ('생채기', 0.7856615781784058),
 ('뜯어내', 0.7800561785697937),
 ('근질거리', 0.7761126756668091),
 ('까맣', 0.7741507291793823),
 ('파먹', 0.7725013494491577)]

In [131]:
model_keywords.wv.similar_by_word("감기")

[('목감기', 0.7787765860557556),
 ('코감기', 0.7744455337524414),
 ('기침감기', 0.730884850025177),
 ('편도염', 0.7144458889961243),
 ('냉방병', 0.7078796029090881),
 ('편도선염', 0.7012592554092407),
 ('열나', 0.6904149651527405),
 ('인후염', 0.689134955406189),
 ('환절기', 0.6878073215484619),
 ('몸살감기', 0.6837522983551025)]

In [132]:
model_keywords.wv.similar_by_word("열")

[('열기', 0.5812327265739441),
 ('닫', 0.562920331954956),
 ('고열로', 0.5314461588859558),
 ('외감', 0.5308478474617004),
 ('오한', 0.5048243999481201),
 ('식히', 0.49996882677078247),
 ('허열', 0.4967460036277771),
 ('불덩이', 0.4954458177089691),
 ('미열', 0.48844265937805176),
 ('고열', 0.48239243030548096)]

In [133]:
model_keywords.wv.similar_by_word("아파")

[('어린애', 0.8474108576774597),
 ('참기', 0.8298739194869995),
 ('아파하', 0.8275489807128906),
 ('쓰라리', 0.8170633316040039),
 ('발버둥치', 0.8054256439208984),
 ('놀래', 0.8050146102905273),
 ('생채기', 0.8005754947662354),
 ('겁나', 0.7973811626434326),
 ('우습', 0.7967796325683594),
 ('거려', 0.7963449954986572)]

In [134]:
model_keywords.wv.similar_by_word("울음")

[('우', 0.7023470401763916),
 ('신음', 0.6802157759666443),
 ('울', 0.6786100268363953),
 ('어르', 0.6770550012588501),
 ('자지러지', 0.674082338809967),
 ('칭얼거리', 0.6679117679595947),
 ('흐느끼', 0.6657028198242188),
 ('큰소리', 0.657406210899353),
 ('보채', 0.6568694710731506),
 ('괴성', 0.656578779220581)]

In [135]:
model_keywords.wv.similar_by_word("남아", topn=20)

[('여아', 0.807877779006958),
 ('정상아', 0.6655420064926147),
 ('성징', 0.6519114375114441),
 ('확연', 0.6414461135864258),
 ('기라', 0.6359351277351379),
 ('두드러지', 0.631649374961853),
 ('남학생', 0.6231328248977661),
 ('색맹', 0.6228243112564087),
 ('생시', 0.6227834820747375),
 ('출생체중', 0.6215988993644714),
 ('자폐아', 0.6194701194763184),
 ('증후', 0.612797737121582),
 ('조숙', 0.610781192779541),
 ('가슴둘레', 0.6084136962890625),
 ('극소', 0.6076409816741943),
 ('위수', 0.6069537401199341),
 ('정상인', 0.6066635847091675),
 ('정류', 0.6064138412475586),
 ('보인자', 0.6047580242156982),
 ('흉위', 0.6046409606933594)]

In [136]:
model_keywords.wv.similar_by_word("여아", topn=20)

[('남아', 0.807877779006958),
 ('가슴둘레', 0.5910583734512329),
 ('생시', 0.5680595636367798),
 ('성징', 0.5669124126434326),
 ('남학생', 0.5508815050125122),
 ('기라', 0.5487903356552124),
 ('곱슬머리', 0.5473015308380127),
 ('성별', 0.5472237467765808),
 ('흉위', 0.5462855100631714),
 ('여자', 0.5404715538024902),
 ('위수', 0.536983847618103),
 ('여학생', 0.536203145980835),
 ('정상아', 0.5322072505950928),
 ('남자', 0.5308912992477417),
 ('여탕', 0.5266007781028748),
 ('극소', 0.5220505595207214),
 ('확연', 0.521916389465332),
 ('출생체중', 0.5213947892189026),
 ('아도', 0.5200939178466797),
 ('남근', 0.519985556602478)]

In [137]:
model_keywords.wv.similar_by_word("여자", topn=20)

[('남자', 0.8688820600509644),
 ('금성', 0.5735706686973572),
 ('여탕', 0.5641278624534607),
 ('드세', 0.5638848543167114),
 ('남아', 0.5595381259918213),
 ('조신', 0.543067216873169),
 ('여아', 0.5404715538024902),
 ('남학생', 0.5300654172897339),
 ('화성인', 0.529469907283783),
 ('짓궂', 0.5271279811859131),
 ('아이든', 0.5264469981193542),
 ('남탕', 0.5231404900550842),
 ('속사정', 0.5203368663787842),
 ('성차', 0.5145344138145447),
 ('확연', 0.5138534307479858),
 ('족쇄', 0.5120424032211304),
 ('얌전', 0.508198618888855),
 ('처갓집', 0.5072363615036011),
 ('후예', 0.5035378932952881),
 ('엘렉트라', 0.5021142363548279)]

In [138]:
model_keywords.wv.similar_by_word("남자", topn=20)

[('여자', 0.8688820600509644),
 ('금성', 0.6229436993598938),
 ('여탕', 0.6108936071395874),
 ('드세', 0.6104174852371216),
 ('조신', 0.5911350250244141),
 ('남아', 0.5884765982627869),
 ('얌전', 0.5823819041252136),
 ('족쇄', 0.5750948190689087),
 ('아이든', 0.5748463869094849),
 ('속사정', 0.5699074268341064),
 ('위엄', 0.5656229853630066),
 ('들추', 0.562061071395874),
 ('확연', 0.5614979267120361),
 ('남학생', 0.5613505840301514),
 ('짓궂', 0.5607621073722839),
 ('남탕', 0.5586973428726196),
 ('알파걸', 0.5500965714454651),
 ('객체', 0.5498368144035339),
 ('바깥일', 0.5454378724098206),
 ('성차', 0.5448817610740662)]

In [139]:
model_keywords.wv.similar_by_word("유모차")

[('디럭스', 0.731662392616272),
 ('절충', 0.6923345327377319),
 ('타보', 0.6920753717422485),
 ('잉글', 0.6908161044120789),
 ('휴대용', 0.6686482429504395),
 ('레시', 0.6642338037490845),
 ('핸들링', 0.6603004932403564),
 ('포투', 0.64597487449646),
 ('퀴니', 0.6429663896560669),
 ('맥클라렌', 0.6423449516296387)]

In [140]:
model_keywords.wv.most_similar(positive=["중고", "유모차"])

[('내구재', 0.7264575958251953),
 ('휴대용', 0.7040308713912964),
 ('범용', 0.6971195340156555),
 ('맥클라렌', 0.6886078715324402),
 ('미니밴', 0.6810783743858337),
 ('타보', 0.6806071996688843),
 ('중저가', 0.6802986860275269),
 ('절충', 0.6762381792068481),
 ('삼륜', 0.6755664348602295),
 ('소비재', 0.6746143698692322)]

In [141]:
model_keywords.wv.most_similar(positive=["좋", "유모차"])

[('체크포인트', 0.6493660807609558),
 ('기동력', 0.646479606628418),
 ('니라', 0.6446662545204163),
 ('삼륜', 0.6420556306838989),
 ('봄나들이', 0.6385335922241211),
 ('다단', 0.6377896070480347),
 ('행선지', 0.6374572515487671),
 ('플루토', 0.6313495635986328),
 ('라부', 0.6298192739486694),
 ('눈독', 0.6292997002601624)]

In [142]:
model_keywords.wv.most_similar(positive=["안", "좋", "유모차"])

[('머미', 0.7009689807891846),
 ('차광막', 0.6974648833274841),
 ('무거우', 0.6946828365325928),
 ('침대위', 0.6911909580230713),
 ('니라', 0.6894622445106506),
 ('다단', 0.6892098784446716),
 ('이동도', 0.6889286041259766),
 ('가슴골', 0.686928927898407),
 ('껄끄럽', 0.6864690184593201),
 ('비켜나', 0.6861744523048401)]

In [143]:
model_keywords.wv.most_similar(positive=["안", "좋"])

[('가슴골', 0.6530451774597168),
 ('목마르', 0.6516422629356384),
 ('차면', 0.6494159698486328),
 ('아달라', 0.6397186517715454),
 ('놀래', 0.6353753209114075),
 ('어두우', 0.6342929005622864),
 ('거하', 0.6337921619415283),
 ('머미', 0.633304238319397),
 ('쪼이', 0.6331221461296082),
 ('내던지', 0.6319912672042847)]

In [147]:
model_keywords.wv.most_similar(positive=["가슴골"])

[('온습포', 0.9232149124145508),
 ('수온계', 0.9103384613990784),
 ('요탕', 0.9027554988861084),
 ('쳐지', 0.8979514837265015),
 ('욕법', 0.8939695358276367),
 ('젖멍울', 0.8939502239227295),
 ('삼각근', 0.8900146484375),
 ('물수', 0.886853814125061),
 ('머미', 0.886379599571228),
 ('받쳐들', 0.8859356641769409)]

---

### Volume of Corpus 

In [63]:
df[['text']].to_csv('data/text_for_word2vec.csv', index=False, encoding='utf-8')

In [64]:
pd.read_csv('data/text_for_word2vec.csv')

,text
0,6살 남아 입니다 회사 복직 후 3살부터 주중에는 외할머니댁에서 2살 4살 터울의 ...
1,요즘 들어 아빠가 아이에게 다가가려고 하면 엄마한테 갈 거야 하면서 엄마를 유독 찾...
2,둘째가 태어나면 첫째가 힘들 거라는 건 예상하고 있었지만 툭하면 삐치고 울고 좀처럼...
3,20개월 남아가 있고 임신 8개월차로 두 아기의 엄마입니다 첫 아이는 잠자리 독립을...
4,14개월 남자 아기인데 심하게 깨무는 버릇이 있어요 전에는 졸릴 때 주로 물곤 했는...
...,...
25344,가임기 연력 여성의 흡연률 증가와 그 폐해 현대 여성은 다양한 사회활동에 참여하고 ...
25345,아토피에 유전적인 요인이 작용한다는 사실 알고 계시나요 유전적인 요인 뿐 아니라 아...
25346,1st 시기에 따라 필요한 영양소를 충분히 뱃속의 아이는 엄마가 먹는 것을 함께 먹...
25347,분만 여성만이 누릴 수 있는 특권 대부분의 여성들은 임신과 분만이라는 과정을 경험하...


---

## Doc2Vec

In [65]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [66]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(df['noun'])]

In [67]:
%%time
doc2vec_model = Doc2Vec(documents, 
                        vector_size=200, 
                        window=2, 
                        min_count=5, 
                        workers=4)


doc2vec_model.save('data/tokmom_doc2vec_only_noun.model')

CPU times: user 1min 10s, sys: 1.26 s, total: 1min 11s
Wall time: 29.4 s


In [68]:
len(doc2vec_model.docvecs)

25349

In [69]:
df['text'][0]

'6살 남아 입니다 회사 복직 후 3살부터 주중에는 외할머니댁에서 2살 4살 터울의 사촌 형 누나와 함께 자랐어요 금요일 저녁에 데려와 주말에는 엄마 아빠와 생활하고 있구요 7세가 되는 내년이라도 아이를 데려오자는 아이 아빠의 의견과 유치원과 태권도 등 패턴이 적응되었고 형 누나들과 더 자랄 수 있게 초등학교 입학시기에 데리고 가는게 낫지 않겠냐는 외할머니 의견이 다릅니다 6세가 된 아이의 교육과 생활습관 형성이 이제는 할머니댁에서는 어려울 것 같다는 생각이 들고 하루하루 생각이 크고 자라는 아이를 주말에만 보는 저 또한 놀랄 때가 많거든요 하루라도 빨리 데려와야 할까요 초등학교 입학시기에 데려오는 편이 아이에게 좋을까요 매우 어려운 질문입니다 먼저 아이가 현재 잘 적응하고 있고 주변 양육환경 역시 아이가 성장하기에 별다른 문제가 없다 하니 고민이 더욱 크시리라 생각됩니다 우선 되어야하는 것은 아이의 의견입니다 6살이면 이러한 문제에 대하여 스스로 생각하고 의견을 전달할 수 있으므로 먼저 아이의 의견을 들어 주시기 바랍니다 아이는 부모님과 함께 사는 것이 좋습니다 그러나 부모님께서 두분 모두 직장에 나가시니 유치원 하원 후 아이를 돌보아 주실 분이 현재 함께 생활하고 계시는 할머님이시면 재고의 여지 없이 데려 오시는 것을 권유 드립니다 그러나 하원 후 할머님의 돌봄이 여의치 못할 경우 돌보아 줄 대상이 당분간 할머님댁에서 시간제로나마 아이를 돌보아 주어 아이와의 상호작용을 충분히 가진 후에 아이가 집으로 왔을 때 유치원 하원 후에 계속 돌보아 줄 수 있으면 좋습니다 이와 같이 먼저 아이가 집으로 온 후 부모님께서 퇴근하시기 전 까지 돌보아 줄 사람을 알아보시고 데려 오시는 것이 좋다는 생각이 듭니다'

In [70]:
print("Input Document", df['text'][100], sep="\n")

similar_docs = doc2vec_model.docvecs.most_similar(100)

print("Most similar Documents")
for idx, similarity in similar_docs[:3]:
    print("idx:", idx, "similarity:", np.round(similarity, 3))
    print(df['text'][idx])

Input Document
젖병을 떼야 하는 이유로는 첫째 우식증을 예방하고 둘째 젖병 무는 어린 아기의 습관을 서서히 고쳐 나가기 위함 입니다 지금은 단지 습관에 의해 젖병을 물고 있을 뿐이므로 음식은 숟가락으로 먹이고 우유는 컵으로 마시게 하면서 젖병을 뗄 수 있도록 해야 합니다 잠잘 때 젖병을 물고 자는 것은 무의식적인 행동입니다 그러나 너무 친숙해진 습관을 갑자기 금지시킨다면 아기 심리에 큰 충격을 줄 수도 있으므로 서서히 젖병 속의 우유를 희석시키는 방법을 사용합니다 예를 들어 평상시에 우유를 200 mL 먹였다면 우유 180 mL에 물 20 mL를 섞어서 젖병에 넣어줍니다 점차 물의 농도를 높여가다가 아기가 이를 느끼고 부정적인 반응을 보일 때는 잠시 정지하였다가 다시 희석의 강도를 높여봅니다 그러다 보면 아기는 점차 희석된 우유에 흥미를 잃어 스스로 젖병을 내던지게 될 것입니다 아기에 따라 스스로 정서적 안정을 얻는 습관을 하나씩 가지고 있습니다 아기 때의 이불을 늘 잡고 있다거나 엄마의 머리카락을 잡아야 잠을 잔다거나 하는 습관은 고치기가 쉽지 않은 일입니다 젖병도 그런 습관의 일종일 수 있습니다 젖병을 물고 있는 것이 정서적인 이유라면 젖병 떼기를 강요하지 않습니다 하루 2 3번은 젖병을 물고 있게 하다가 점차 아기가 성장하면서 젖병 무는 것이 자신에게는 어울리지 않는다는 것을 엄마가 끊임없이 설득하고 아이 스스로도 그렇게 느껴 자연스레 젖병을 멀리하게 합니다 가능한 컵이나 빨대컵을 활용하고 노리개 젖꼭지 등을 대체하여 사용해 볼 수도 있습니다
Most similar Documents
idx: 15452 similarity: 0.667
아기가 젖병만 보면 입으로 가져가는 이유는 왜일까 배가 고프다는 의사 표현일까 아니면 말랑말랑 젤리 같은 젖꼭지의 느낌이 좋아서일까 혹시 어딘가 아프거나 불만이 있다는 무언의 표현은 아닐까 배가 고프면 엄마를 찾거나 우유를 찾는 것은 당연한 일이겠지만 아기들은 배가 고프지 않아도 젖병을 물고 싶어한다 별다른 이유

In [71]:
df['tokenized_doc'] = df['noun'].apply(lambda x: doc2vec_model.infer_vector(x).astype(float))

In [72]:
df

,text,tokenized,noun,keywords,tokenized_doc
0,6살 남아 입니다 회사 복직 후 3살부터 주중에는 외할머니댁에서 2살 4살 터울의 ...,"[6, 살, 남, 아, 입, 니, 다, 회사, 복직, 후, 3, 살, 부터, 주, ...","[살, 회사, 복직, 후, 살, 주, 중, 외할머니, 댁, 살, 살, 터울, 사촌,...","[남, 입, 회사, 복직, 후, 주, 외할머니, 댁, 터울, 사촌, 형, 누나, 금...","[0.1641332358121872, -4.2163810576312244e-05, ..."
1,요즘 들어 아빠가 아이에게 다가가려고 하면 엄마한테 갈 거야 하면서 엄마를 유독 찾...,"[요즘, 들, 어, 아빠, 가, 아이, 에게, 다가가, 려고, 하, 면, 엄마, 한...","[아빠, 아이, 엄마, 거, 엄마, 전, 정도, 엄마, 아빠, 거부, 엄마, 눈, ...","[들, 아빠, 아이, 다가가, 하, 엄마, 하, 엄마, 찾, 전, 정도, 엄마, 매...","[0.28892576694488525, -0.5387788414955139, 0.1..."
2,둘째가 태어나면 첫째가 힘들 거라는 건 예상하고 있었지만 툭하면 삐치고 울고 좀처럼...,"[둘째, 가, 태어나, 면, 첫째, 가, 힘들, 거, 라는, 건, 예상, 하, 고,...","[둘째, 첫째, 거, 건, 예상, 때, 엄마, 껌, 딱지, 첫째, 외출, 때, 제외...","[태어나, 예상, 삐치, 울, 서럽, 웁, 때, 엄마, 껌, 딱지, 외출, 때, 제...","[0.08370722085237503, -0.13255001604557037, -0..."
3,20개월 남아가 있고 임신 8개월차로 두 아기의 엄마입니다 첫 아이는 잠자리 독립을...,"[20, 개월, 남아, 가, 있, 고, 임신, 8, 개월, 차, 로, 두, 아기, ...","[개월, 남아, 임신, 개월, 차, 아기, 엄마, 아이, 잠자리, 독립, 생후, 일...","[남아, 있, 임신, 차, 아기, 엄마, 아이, 잠자리, 독립, 생후, 전, 재우,...","[0.17814810574054718, -0.2249746322631836, 0.2..."
4,14개월 남자 아기인데 심하게 깨무는 버릇이 있어요 전에는 졸릴 때 주로 물곤 했는...,"[14, 개월, 남자, 아기, 인데, 심하, 게, 깨무, 는, 버릇, 이, 있, 어...","[개월, 남자, 아기, 버릇, 전, 때, 이제, 자기, 지대, 때, 떼, 상대방, ...","[남자, 아기, 심하, 깨무, 버릇, 있, 전, 때, 물, 이제, 지대, 때, 떼,...","[-0.029396692290902138, 0.1356123685836792, -0..."
...,...,...,...,...,...
25344,가임기 연력 여성의 흡연률 증가와 그 폐해 현대 여성은 다양한 사회활동에 참여하고 ...,"[가, 임기, 연력, 여성, 의, 흡연, 률, 증가, 와, 그, 폐해, 현대, 여성...","[임기, 연력, 여성, 흡연, 증가, 폐해, 현대, 여성, 사회, 활동, 참여, 경...","[임기, 연력, 여성, 흡연, 증가, 폐해, 현대, 여성, 사회, 활동, 참여, 경...","[0.10278913378715515, 0.2534139156341553, -0.1..."
25345,아토피에 유전적인 요인이 작용한다는 사실 알고 계시나요 유전적인 요인 뿐 아니라 아...,"[아토피, 에, 유전, 적, 인, 요인, 이, 작용, 한다는, 사실, 알, 고, 계...","[아토피, 유전, 요인, 작용, 사실, 유전, 요인, 뿐, 아토피, 유발, 수, 하...","[아토피, 유전, 요인, 작용, 사실, 알, 유전, 요인, 아토피, 유발, 있, 중...","[0.04924286901950836, 0.07227320224046707, 0.0..."
25346,1st 시기에 따라 필요한 영양소를 충분히 뱃속의 아이는 엄마가 먹는 것을 함께 먹...,"[1, st, 시기, 에, 따라, 필요, 한, 영양소, 를, 충분히, 뱃속, 의, ...","[시기, 필요, 영양소, 뱃속, 아이, 엄마, 것, 균형, 영양, 음식, 섭취, 태...","[시기, 필요, 영양소, 뱃속, 아이, 엄마, 먹, 먹, 균형, 잡히, 영양, 있,...","[0.2840946614742279, -0.03573429957032204, -0...."
25347,분만 여성만이 누릴 수 있는 특권 대부분의 여성들은 임신과 분만이라는 과정을 경험하...,"[분만, 여성, 만, 이, 누릴, 수, 있, 는, 특권, 대부분, 의, 여성, 들,...","[분만, 여성, 수, 특권, 대부분, 여성, 임신, 분만, 과정, 경험, 삶, 의미...","[분만, 여성, 있, 특권, 대부분, 여성, 임신, 분만, 과정, 경험, 삶, 의미...","[0.1448511928319931, 0.10906893759965897, -0.1..."
